

# Grounding with Bing
**Grounding with Bing Search** allows your Azure AI Agents to incorporate real-time public web data when generating responses. You need to create a Grounding with Bing Search resource, and then connect this resource to your Azure AI Agents. When a user sends a query, Azure AI Agents decide if Grounding with Bing Search should be leveraged or not. If so, it will leverage Bing to search over public web data and return relevant chunks. Lastly, Azure AI Agents will use returned chunks to generate a response.  

You can ask questions such as "*what is the top news today*" or "*what is the recent update in the retail industry in the US?*", which require real-time public data.

Developers and end users don't have access to raw content returned from Grounding with Bing Search. The response, however, includes citations with links to the websites used to generate the response, and a link to the Bing query used for the search. These two *References* must be retained and displayed in the exact form provided by Microsoft, as per Grounding with Bing Search's [Use and Display Requirements](https://www.microsoft.com/en-us/bing/apis/grounding-legal#use-and-display-requirements). See the [how to display Grounding with Bing Search results](#how-to-display-grounding-with-bing-search-results) section for details.

>[!IMPORTANT]
> 1. Your usage of Grounding with Bing Search may incur costs. See the [pricing page](https://www.microsoft.com/bing/apis/grounding-pricing) for details.
> 1. By creating and using a Grounding with Bing Search resource through code-first experience, such as Azure CLI, or deploying through deployment template, you agree to be bound by and comply with the terms available at https://www.microsoft.com/en-us/bing/apis/grounding-legal, which may be updated from time to time.


## **Setup:**  

> [!NOTE]
> 1. Grounding with Bing Search only works with the following Azure OpenAI models: `gpt-3.5-turbo-0125`, `gpt-4-0125-preview`, `gpt-4-turbo-2024-04-09`, `gpt-4o-0513`

1. Register the Bing Search provider.

   ```console
       az provider register --namespace 'Microsoft.Bing'
   ```

1. Create a new Grounding with Bing Search resource in the [Azure portal](https://portal.azure.com/#create/Microsoft.BingGroundingSearch), and select the different fields in the creation form. Make sure you create this Grounding with Bing Search resource in the same resource group as your Azure AI Agent, AI Project, and other resources.

1. After you have created a Grounding with Bing Search resource, you can find it in [Azure portal](https://portal.azure.com/#home). Navigate to the resource group you've created the resource in, search for the Grounding with Bing Search resource you have created.

   ![alt text](assets/img/resource-azure-portal.png)

1. Select the Grounding with Bing Search resource you have created and copy any of the API keys.

    ![alt text](assets/img/key-resource-azure-portal.png)

1. Go to the [Azure AI Foundry portal](https://ai.azure.com/) and select the AI Project (make sure it's in the same resource group of your Grounding with Bing Search resource). Click **management center**.

    ![alt text](assets/img/project-settings-button.png)

1. Select **+ new connection** in the settings page. 

    >[!NOTE]
    > If you re-generate the API key at a later date, you need to update the connection with the new key.

    ![alt text](assets/img/project-connections.png)

1. Select **API key** in **other resource types**.

    ![alt text](assets/img/api-key-connection.png)

1. Enter the following information and then create a new connection to your Grounding with Bing Search resource.

    - Endpoint: `https://api.bing.microsoft.com/`
    - Key: `YOUR_API_KEY`
    - Connection name: `YOUR_CONNECTION_NAME` (You will use this connection name in the sample code below.)
    - Access: you can choose either *this project only* or *shared to all projects*. Just make sure in the sample code below, the project you entered connection string for has access to this connection.

Now that we have configured everything, let's create a client object, which will contain the connection string for connecting to your AI project and other resources. To make the Grounding with Bing search tool available to your agent, use a connection to initialize the tool and attach it to the agent. You can find your connection in the **connected resources** section of your project in the Azure AI Foundry portal.



In [1]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import BingGroundingTool


# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

In [2]:
bing_connection = project_client.connections.get(
    connection_name=os.environ["BING_CONNECTION_NAME"]
)
conn_id = bing_connection.id

print(conn_id)

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

# Create agent with the bing tool and process assistant run
project_client.__enter__()
agent = project_client.agents.create_agent(
    model="gpt-4-turbo",
    name="my-assistant",
    instructions="You are a helpful assistant",
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"}
)
print(f"Created agent, ID: {agent.id}")

/subscriptions/3490bb6d-9e8c-46c5-b635-f8fea8df3d69/resourceGroups/agent-quickstart-rg2/providers/Microsoft.MachineLearningServices/workspaces/project-demo-agents-oaic/connections/bing_grounding
Created agent, ID: asst_9nMPjXiesA2UFWP8rPGjmF3F


In [3]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="What is the top news today",
)
print(f"Created message, ID: {message.id}")

Created thread, ID: thread_nvz9nRB6xt0WMoiFMdvexjaD
Created message, ID: msg_Z08PxBv6qPm3qWNfmY9iKoOd


In [4]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}")
print(f"Run ID: {run.id}")

# Retrieve run step details to get Bing Search query link
# To render the webpage, we recommend you replace the endpoint of Bing search query URLs with `www.bing.com` and your Bing search query URL would look like "https://www.bing.com/search?q={search query}"
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
print(f"Last run step detail: {run_steps_data}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")


messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

Run finished with status: completed
Run ID: run_SUwSsszYEK4EnTiLLuIxdJ0Q
Last run step detail: [{'id': 'step_oYnFq52ZF1XWQ7IZAuRMdaRQ', 'object': 'thread.run.step', 'created_at': 1736934865, 'run_id': 'run_SUwSsszYEK4EnTiLLuIxdJ0Q', 'assistant_id': 'asst_9nMPjXiesA2UFWP8rPGjmF3F', 'thread_id': 'thread_nvz9nRB6xt0WMoiFMdvexjaD', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1736934870, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_WGds8HjJwNFgbNPae8IIWKtI'}}, 'usage': {'prompt_tokens': 1659, 'completion_tokens': 197, 'total_tokens': 1856}}, {'id': 'step_9spIN6KL3FVXS3042aLzkFHF', 'object': 'thread.run.step', 'created_at': 1736934864, 'run_id': 'run_SUwSsszYEK4EnTiLLuIxdJ0Q', 'assistant_id': 'asst_9nMPjXiesA2UFWP8rPGjmF3F', 'thread_id': 'thread_nvz9nRB6xt0WMoiFMdvexjaD', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'completed_a

In [5]:
# Delete the agent once done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")
project_client.__exit__(None, None, None)

Deleted agent


> **Important Considerations:** 
>    * According to Grounding with Bing's [terms of use and use and display requirements](https://www.microsoft.com/en-us/bing/apis/grounding-legal#use-and-display-requirements), you need to display both website URLs and Bing search query URLs in your custom interface. You can find website URLs through `annotations` parameter in API response and Bing search query URLs through `runstep` details. To render the webpage, we recommend you replace the endpoint of Bing search query URLs with `www.bing.com` and your Bing search query URL would look like "https://www.bing.com/search?q={search query}"
>    * Microsoft will use data you send to Grounding with Bing to improve Microsoft products and services. Where you send personal data to this service, you are responsible for obtaining sufficient consent from the data subjects. The Data Protection Terms in the Online Services Terms do not apply to Grounding with Bing. 


# Grounding with file to assistant API

The file search tool uses the Azure AI Search and Azure Blob Storage resources you connected during agent setup.

* Uploaded files get stored in your connected Azure Blob Storage account
* Vector stores get created using your connected Azure AI Search resource

For both agent setups, Azure OpenAI handles the entire ingestion process, which includes:

* Automatically parsing and chunking documents
* Generating and storing embeddings
* Utilizing both vector and keyword searches to retrieve relevant content for user queries.
There is no difference in the code between the two setups; the only variation is in where your files and created vector stores are stored.

## How it works
The file search tool implements several retrieval best practices out of the box to help you extract the right data from your files and augment the model’s responses. The file search tool:

* Rewrites user queries to optimize them for search.
* Breaks down complex user queries into multiple searches it can run in parallel.
* Runs both keyword and semantic searches across both agent and thread vector stores.
* Reranks search results to pick the most relevant ones before generating the final response.
* By default, the file search tool uses the following settings:
    * Chunk size: 800 tokens
    * Chunk overlap: 400 tokens
    * Embedding model: text-embedding-3-large at 256 dimensions
    * Maximum number of chunks added to context: 20

## Vector stores
Vector store objects give the file search tool the ability to search your files. Adding a file to a vector store automatically parses, chunks, embeds, and stores the file in a vector database that's capable of both keyword and semantic search. Each vector store can hold up to 10,000 files. Vector stores can be attached to both agents and threads. Currently you can attach at most one vector store to an agent and at most one vector store to a thread.

Similarly, these files can be removed from a vector store by either:

* Deleting the vector store file object or,
* By deleting the underlying file object, which removes the file it from all vector_store and code_interpreter configurations across all agents and threads in your organization
The maximum file size is 512 MB. Each file should contain no more than 5,000,000 tokens per file (computed automatically when you attach a file).

## Ensuring vector store readiness before creating runs
We highly recommend that you ensure all files in a vector_store are fully processed before you create a run. This ensures that all the data in your vector store is searchable. You can check for vector store readiness by using the polling helpers in the SDKs, or by manually polling the vector store object to ensure the status is completed.

As a fallback, there's a 60-second maximum wait in the run object when the thread's vector store contains files that are still being processed. This is to ensure that any files your users upload in a thread a fully searchable before the run proceeds. This fallback wait does not apply to the agent's vector store.

## Quickstart – Upload Local Files with file search
In this example, we use Azure AI Agent Service to create an agent that can help answer questions on information you upload from local files.

### Prerequisites
Complete the agent setup.

* Ensure that you have the role Storage Blob Data Contributor on your project's storage account.
* Ensure that you have the role Azure AI Developer on your project.

### Create your agent
Make sure that once you have created the project client you upload your file and create a vector store using its *file.id*. Once that has been done create your agent and add the file search tool definitions and resources respectively to *tools* and to *tool_resources* 

In [13]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FileSearchTool, MessageAttachment, FilePurpose
from azure.identity import DefaultAzureCredential


# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# At the moment, it should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<ProjectName>"
# Customer needs to login to Azure subscription via Azure CLI and set the environment variables

credential = DefaultAzureCredential()
project_client = AIProjectClient.from_connection_string(
    credential=credential, conn_str=os.environ["PROJECT_CONNECTION_STRING"] 
)
# We will upload the local file and will use it for vector store creation.

# upload a file
file = project_client.agents.upload_file_and_poll(file_path='./assets/data/product_info.md', purpose=FilePurpose.AGENTS)
print(f"Uploaded file, file ID: {file.id}")

# create a vector store with the file you uploaded
vector_store = project_client.agents.create_vector_store_and_poll(file_ids=[file.id], name="my_vectorstore")
print(f"Created vector store, vector store ID: {vector_store.id}")
# create a file search tool
file_search_tool = FileSearchTool(vector_store_ids=[vector_store.id])

# notices that FileSearchTool as tool and tool_resources must be added or the agent will be unable to search the file
agent = project_client.agents.create_agent(
    model="gpt-4-turbo",
    name="my-agent",
    instructions="You are a helpful agent",
    tools=file_search_tool.definitions,
    tool_resources=file_search_tool.resources,
)
print(f"Created agent, agent ID: {agent.id}")

thread_grounding = project_client.agents.create_thread()
message = project_client.agents.create_message(
    thread_id=thread_grounding.id,
    role="user",
    content="I have an issue with the tent poles. Are those covered by the waranty?",
)

 # Run the agent
run = project_client.agents.create_and_process_run(thread_id=thread_grounding.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

# Get messages from the thread
messages = project_client.agents.list_messages(thread_id=thread_grounding.id)
print(f"Messages: {messages}")


project_client.agents.delete_agent(agent.id)
print("Deleted agent")


Uploaded file, file ID: assistant-XwPTMOm7HxwJa52FDYtJHAHQ
Created vector store, vector store ID: vs_8ZEELrWkYkrFEQYH9vHYKbdG
Created agent, agent ID: asst_syY2GY88obR5kL9PDuZL7p4N
Run finished with status: completed
Messages: {'object': 'list', 'data': [{'id': 'msg_6FT0sc81mOiIrP2U6WXDfpuB', 'object': 'thread.message', 'created_at': 1737019793, 'assistant_id': 'asst_syY2GY88obR5kL9PDuZL7p4N', 'thread_id': 'thread_rBbeeUxIFtO8OJOp8EAVceEt', 'run_id': 'run_XDLuMYQsEACd3ba1173Ag9It', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'Yes, issues with tent poles are covered by the warranty provided they arise from normal use and are not due to misuse or abuse. Specifically, the warranty for the tent includes coverage for "issues with the tent poles, zippers, buckles, or other hardware components that affect the functionality of the tent"【4:0†source】.', 'annotations': [{'type': 'file_citation', 'text': '【4:0†source】', 'start_index': 308, 'end_index': 320, 'file_citation':

Another way this can be done is by attaching the files as Message attachments on your thread. Doing so creates another vector_store associated with the thread, or, if there's already a vector store attached to this thread, attaches the new files to the existing thread vector store. When you create a Run on this thread, the file search tool queries both the vector_store from your agent and the vector_store on the thread.
```python
# Create a thread
thread = project_client.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

# Upload the user provided file as a messsage attachment
message_file = project_client.agents.upload_file_and_poll(file_path='product_info_1.md', purpose=FilePurpose.AGENTS)
print(f"Uploaded file, file ID: {message_file.id}")

# Create a message with the file search attachment
# Notice that vector store is created temporarily when using attachments with a default expiration policy of seven days.
attachment = MessageAttachment(file_id=message_file.id, tools=FileSearchTool().definitions)
message = project_client.agents.create_message(
    thread_id=thread.id, role="user", content="What feature does Smart Eyewear offer?", attachments=[attachment]
)
print(f"Created message, message ID: {message.id}")
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Created run, run ID: {run.id}")
```
Try it for your self